## 1. Import Libraries

In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

get_ipython().system('pip install folium')
import folium
print('Libraries imported.')

Libraries imported.


## 2. Scrap data from Wikipedia page into a DataFrame

In [13]:
url='https://en.wikipedia.org/wiki/Boroughs_and_quarters_of_Hamburg'
wiki=requests.get(url).text


In [14]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(wiki, 'html.parser')

In [15]:
# create a list to store neighborhood data
neighborhoodList = []

In [17]:
# append the data into the list
for row in soup.find_all("div", class_="refbegin reflist columns references-column-width")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [19]:
df = pd.DataFrame({"Neighborhood": neighborhoodList})

df.head()

,Neighborhood
0,Allermöhe
1,Alsterdorf
2,Altengamme
3,Altenwerder
4,Altona-Altstadt


In [20]:
# print the number of rows of the dataframe
df.shape

(104, 1)

## 3. Get the geographical coordinates

In [21]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hamburg, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [29]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]

In [33]:
# populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df_coords.head()

,Latitude,Longitude
0,53.49879,10.11042
1,53.60851,10.00995
2,53.42756,10.26881
3,53.51193,9.91553
4,53.55078,9.94071


In [36]:
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df.head()

(104, 3)


,Neighborhood,Latitude,Longitude
0,Allermöhe,53.49879,10.11042
1,Alsterdorf,53.60851,10.00995
2,Altengamme,53.42756,10.26881
3,Altenwerder,53.51193,9.91553
4,Altona-Altstadt,53.55078,9.94071


In [37]:
# save the DataFrame as CSV file
df.to_csv("hamburg_coordinates_df.csv", index=False)

## 4. Create a map of Hamburg

In [38]:
# get the coordinates of Hamburg
address = 'Hamburg, Germany'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hamburg, Germany {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hamburg, Germany 53.5437641, 10.0099133.


In [39]:
# create map of Hamburg using latitude and longitude values
map_ham = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ham)  
    
map_ham

In [40]:
map_ham.save('hamburg_map.html')

## 5. Use Foursquare API to explore the neighborhoods

In [120]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [42]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [43]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5568, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Allermöhe,53.49879,10.11042,Wutzrock,53.483274,10.110406,Event Space
1,Allermöhe,53.49879,10.11042,IKEA Restaurant,53.510878,10.093949,Scandinavian Restaurant
2,Allermöhe,53.49879,10.11042,Zum Eichbaum,53.487763,10.104665,German Restaurant
3,Allermöhe,53.49879,10.11042,IKEA,53.510870,10.093961,Furniture / Home Store
4,Allermöhe,53.49879,10.11042,IKEA Hot Dog Station,53.511337,10.093385,Hot Dog Joint


###  Let's check how many venues were returned for each neighorhood

In [44]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Allermöhe,22,22,22,22,22,22
Alsterdorf,100,100,100,100,100,100
Altengamme,4,4,4,4,4,4
Altenwerder,11,11,11,11,11,11
Altona-Altstadt,100,100,100,100,100,100
Altona-Nord,100,100,100,100,100,100
Bahrenfeld,100,100,100,100,100,100
Barmbek-Nord,100,100,100,100,100,100
Barmbek-Süd,100,100,100,100,100,100


### Let's find out the unique categories

In [45]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 305 uniques categories.


In [46]:
venues_df['VenueCategory'].unique()[:20]

array(['Event Space', 'Scandinavian Restaurant', 'German Restaurant',
       'Furniture / Home Store', 'Hot Dog Joint', 'Gas Station',
       'Fast Food Restaurant', 'Hardware Store', 'Bus Stop',
       'Performing Arts Venue', 'Light Rail Station', 'Business Service',
       'Soccer Field', 'Tunnel', 'Boat Rental', 'Intersection',
       'Racetrack', 'Lake', 'Bakery', 'River'], dtype=object)

In [50]:
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

## 6. Neighborhood Analysis

In [51]:
# one hot encoding
ham_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ham_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ham_onehot.columns[-1]] + list(ham_onehot.columns[:-1])
ham_onehot = ham_onehot[fixed_columns]

print(ham_onehot.shape)
ham_onehot.head()

(5568, 306)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paella Restaurant,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stables,Steakhouse,Storage Facility,Supermarket,Surf Spot,Sushi Restaurant,Swabian Restaurant,Szechuan Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,

###  Group rows by neighborhood

In [52]:
ham_grouped = ham_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(ham_grouped.shape)
ham_grouped

(104, 306)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bavarian Restaurant,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Kumpir Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Outlet Store,Paella Restaurant,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Playground,Plaza,Pool,Pool Hall,Port,Portuguese Restaurant,Post Office,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stables,Steakhouse,Storage Facility,Supermarket,Surf Spot,Sushi Restaurant,Swabian Restaurant,Szechuan Restaurant,Tanning Salon,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,

### Create a new DataFrame for Shopping Mall data only

In [55]:
ham_mall=ham_grouped[["Neighborhoods","Shopping Mall"]]
ham_mall.head()

,Neighborhoods,Shopping Mall
0,Allermöhe,0.0
1,Alsterdorf,0.0
2,Altengamme,0.0
3,Altenwerder,0.0
4,Altona-Altstadt,0.0


## 7. Clustering Neighborhoods using K-Means

### Run k-means to cluster the neighborhoods in Hamburg into 3 clusters.

In [89]:
kclusters = 3

ham_clustering = ham_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ham_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0])

In [107]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
ham_merged = ham_mall.copy()

# add clustering labels
ham_merged["Cluster Labels"] = kmeans.labels_
ham_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
ham_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Allermöhe,0.0,1
1,Alsterdorf,0.0,1
2,Altengamme,0.0,1
3,Altenwerder,0.0,1
4,Altona-Altstadt,0.0,1


In [108]:
ham_merged

,Neighborhood,Shopping Mall,Cluster Labels
0,Allermöhe,0.000000,1
1,Alsterdorf,0.000000,1
2,Altengamme,0.000000,1
3,Altenwerder,0.000000,1
4,Altona-Altstadt,0.000000,1
5,Altona-Nord,0.000000,1
6,Bahrenfeld,0.000000,1
7,Barmbek-Nord,0.010000,0
8,Barmbek-Süd,0.000000,1
9,Bergedorf,0.017857,0


In [109]:
# add latitude/longitude for each neighborhood
ham_merged= pd.merge(ham_merged, df, on='Neighborhood')
ham_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Allermöhe,0.0,1,53.49879,10.11042
1,Alsterdorf,0.0,1,53.60851,10.00995
2,Altengamme,0.0,1,53.42756,10.26881
3,Altenwerder,0.0,1,53.51193,9.91553
4,Altona-Altstadt,0.0,1,53.55078,9.94071


In [111]:
ham_merged.dtypes

Neighborhood       object
Shopping Mall     float64
Cluster Labels      int32
Latitude          float64
Longitude         float64
dtype: object

### Now, let's visualize the resulting clusters

In [112]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ham_merged['Latitude'], ham_merged['Longitude'], ham_merged['Neighborhood'], ham_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [113]:
map_clusters.save('map_clusters.html')

## Examine Clusters

### Cluster 0

In [114]:
ham_merged.loc[ham_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
7,Barmbek-Nord,0.010000,0,53.58746,10.04661
9,Bergedorf,0.017857,0,53.49015,10.20881
12,Billstedt,0.025000,0,53.53958,10.10324
16,Bramfeld,0.029412,0,53.61305,10.07916
29,Fuhlsbüttel,0.012821,0,53.63456,10.01627
31,Groß Flottbek,0.013699,0,53.56564,9.87784
33,HafenCity,0.010000,0,53.54444,9.99762
34,Hamburg-Altstadt,0.010000,0,53.54992,9.99893
36,Hammerbrook,0.010000,0,53.54623,10.02416
37,Harburg,0.017241,0,53.46087,9.97842


### Cluster 2

In [116]:
ham_merged.loc[ham_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Allermöhe,0.0,1,53.49879,10.11042
1,Alsterdorf,0.0,1,53.60851,10.00995
2,Altengamme,0.0,1,53.42756,10.26881
3,Altenwerder,0.0,1,53.51193,9.91553
4,Altona-Altstadt,0.0,1,53.55078,9.94071
5,Altona-Nord,0.0,1,53.56804,9.94814
6,Bahrenfeld,0.0,1,53.56466,9.90166
8,Barmbek-Süd,0.0,1,53.57473,10.03235
10,Bergstedt,0.0,1,53.67102,10.12325
11,Billbrook,0.0,1,53.52929,10.07583


### Cluster 2

In [117]:
ham_merged.loc[ham_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
26,Farmsen-Berne,0.034483,2,53.60504,10.11892
32,Gut Moor,0.040816,2,53.45202,10.00838
55,Lurup,0.041667,2,53.59130,9.87293
57,Marmstorf,0.047619,2,53.43991,9.96058
71,Osdorf,0.042553,2,53.57398,9.84676
83,Sinstorf,0.076923,2,53.42435,9.97636
